# Оценка качества воды (1 layer perceptron, sigmoid)

описание задачи в файле Screenshot 2022-10-27 at 22.31.14.png

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from IPython.display import Image
from IPython.core.display import HTML 
from sklearn.model_selection import train_test_split
Image(url= "Screenshot 2022-10-27 at 22.31.14.png")

Загружаем данные

In [2]:
data = pd.read_csv("water_quality.csv",encoding='utf-8', sep=';')
data.head(data.shape[0])

,Кислотность,Жесткость,Пригодна для питья?
0,3.0,10.0,Нет
1,11.0,9.8,Нет
2,7.0,9.0,Да
3,8.5,8.9,Да
4,8.3,8.0,Да
5,6.3,6.4,Нет
6,12.0,6.0,Нет
7,7.1,6.0,Да
8,4.0,10.1,Нет
9,6.0,7.0,Нет


Нормализуем

In [3]:
data[data.columns[2]].replace({'Нет': 0, 'Да': 1}, inplace=True)
normalized_df=(data-data.mean())/data.std()
normalized_df[data.columns[2]]=data[data.columns[2]]
X=normalized_df.drop(data.columns[2],axis=1)
y = normalized_df[data.columns[2]]
X

,Кислотность,Жесткость
0,-1.576531,0.585053
1,0.993651,0.483932
2,-0.291440,0.079452
3,0.190469,0.028891
4,0.126214,-0.426149
5,-0.516331,-1.235111
6,1.314923,-1.437351
7,-0.259313,-1.437351
8,-1.255258,0.635613
9,-0.612713,-0.931750


Split Data смотрим на все строки кроме последней

In [4]:
X_train, X_test, y_train, y_test = train_test_split( X.drop(labels=data.shape[0]-1, axis=0), y.drop(labels=data.shape[0]-1, axis=0), test_size=0.33, shuffle=True,random_state=100)
X_val = X.drop(labels=range(0,data.shape[0]-1), axis=0)

Perceptron woth train loop

In [5]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

class Perceptron:

    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.activation_func = sigmoid
        self.weights = None
        self.bias = None


    def fit(self, X, y):
        n_samples, n_features = X.shape

        # init parameters
        self.weights = [0.2,0.5]
        self.bias = 0.1

        y_ = np.where(y > 0 , 1, 0)

        # learn weights
        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = self.activation_func(linear_output)

                # Perceptron update rule
                update = self.lr * y_predicted * (1 - y_predicted) * (y_[idx] - y_predicted)
                
                self.weights += update * x_i
                #print(x_i)
                self.bias += update


    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        y_predicted = np.round(self.activation_func(linear_output))
        return y_predicted

Run train loop

In [6]:
%%time
p = Perceptron(learning_rate=0.01, n_iters=1000)
p.fit(X_train.to_numpy(), y_train.to_numpy())

CPU times: user 102 ms, sys: 29.2 ms, total: 131 ms
Wall time: 107 ms


Делаем прогноз для последней 14-й строки

In [7]:
print(f'Для\n{data.drop(labels=data.columns[2], axis=1).iloc[data.shape[0]-1]}')
print(f'Прогноз: {str(data.columns[2])[:-1]}' if p.predict(X_val.to_numpy())>0.5 else f'Прогноз: не {data.columns[2]}')

Для
Кислотность     8.3
Жесткость      10.3
Name: 13, dtype: float64
Прогноз: Пригодна для питья


Создаем функцию оценки модели

In [8]:
def accuracy(y_true, y_pred):
        accuracy = np.sum(y_true == y_pred) / len(y_true)
        return accuracy

Считаем качество

In [9]:
print("Perceptron classification accuracy", accuracy(y_train.to_numpy(), p.predict(X_train.to_numpy())))

Perceptron classification accuracy 0.875
